In [108]:
# %matplotlib inline
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
from sklearn import decomposition
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class LogisticRegressionSGA:
    def __init__(self, _n=0.01, _e=0.01):
        self._n = _n
        self._e = _e

    def train(self, _X, _Y):
        d = len(_X[0])
        n = len(_X)
        _X = np.insert(_X, 0, [1 for i in range(n)], axis=1)
        t = 0
        w = []
        w.append(np.array([0 for i in range(d+1)]).T)
        temp = np.arange(n)
        np.random.shuffle(temp)
        _X = _X[temp]
        _Y = _Y[temp]

        while True:
            wCopy = w[-1]
            for x, y in zip(_X, _Y):
                gradient = (y-self.sigmoid(np.dot(wCopy, x)))*x
                wCopy = wCopy+self._n*gradient
            w.append(wCopy)
            t += 1
            if np.linalg.norm(w[t]-w[t-1]) <= self._e:
                break
        self._w = w[-1]
        return w[-1]

    def predict(self, X):
        res = []
        for x in X:
            x = np.insert(x, 0, 1, axis=0)
            res.append(self.sigmoid(np.dot(self._w, x)))
        return res

    def sigmoid(self, _z):
        return math.exp(_z)/(1+math.exp(_z))


df = pd.read_csv('iris.csv')
X = df.iloc[:, :4].to_numpy()
Y = df.iloc[:, -1].to_numpy()

pca = decomposition.PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

Ytest = np.where(Y == "virginica", 1, Y)
Ytest = np.where(Ytest != 1, 0, Ytest)

model = LogisticRegressionSGA()
a = model.train(X, Ytest)
Y = np.where(Y == "virginica", "b", Y)
Y = np.where(Y != "b", "r", Y)

fig = plt.figure(figsize=(4, 4))

tempX = list(X[:, 0])
tempY = list(X[:, 1])
tempZ = list(Ytest)

ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=tempX, ys=tempY, zs=tempZ, c=list(Y))
print(a)

a, b, c, d = a[1], a[2], 1, a[0]

x = np.linspace(min(tempX), max(tempX), 30)
y = np.linspace(min(tempY), max(tempY), 30)
c = [[[], []]]
xt = []
for i in range(len(x)):
    xt.append(np.array([x[i], y[i]]))
X, Y = np.meshgrid(x, y)
# Z = (d - a*X - b*Y) / c
Z = np.array(model.predict(xt))/(X*0+1)
surf = ax.plot_surface(X, Y, Z, alpha=0.54)
# ax.set_xlim(min(tempX),max(tempX))
ax.set_xlabel("X")
# ax.set_ylim(min(tempY),max(tempY))
ax.set_ylabel("Y")
# ax.set_zlim(min(tempZ)-0.5,max(tempZ)+0.5)
fig.set_size_inches(10, 5)
ax.view_init(12, -99)

plt.draw()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-6.80410221  5.07009903 -3.29380277]
